In [ ]:
import spacy 
import json

with open(r"D:\hf\code\data_code\spacy_t\scripts\file.json") as f:
    d=json.load(f)
    print(d)


In [ ]:
import json
t_d = ""
queries = []
bdds = []

for record in d:
    for conversation in record['conversations']:
        if conversation['from'] == 'human' and 'query' in conversation:
            queries.append(conversation['query'])
        elif conversation['from'] == 'GPT' and 'BDD' in conversation:
            bdds.append(conversation['BDD'])

with open('queries.txt', 'w') as f:
    for query in queries:
        t_d = t_d + query + '\n'
        f.write(query + '\n')


with open('bdds.txt', 'w') as f:
    for bdd in bdds:
        t_d = t_d + bdd + '\n'
        f.write(bdd + '\n')

In [ ]:
t_d

In [ ]:
t_d = t_d.splitlines()
t_d

In [ ]:
#Import the requisite library
import spacy

#Build upon the spaCy Small Model
nlp = spacy.blank("en")


#Sample text
text = "What Happens When We Select Reports Option from Leave Option and select My Leave Entitlements and usage Report,Select all fields from it & Click on Generate Button"

#Create the EntityRuler
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns
patterns = [
                {"label": "Action", "pattern": "Select"},
                 {"label": "Element", "pattern": "Reports Option"},
                 {"label": "Action", "pattern": "select"},
                 {"label": "Elements", "pattern": "My Leave Entitlements"},
                 {"label": "Action", "pattern": "Click"},
                 {"label": "Element", "pattern": "Generate Button"}
            ]

ruler.add_patterns(patterns)

doc = nlp(text)

#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

In [ ]:
#Import the requisite library
import spacy

#Build upon the spaCy Small Model
nlp = spacy.blank("en")


#Sample text
text = t_d
#Create the EntityRuler
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns
patterns = [
                {"label": "Action", "pattern": "Select"},
                 {"label": "Element", "pattern": "Reports Option"},
                 {"label": "Action", "pattern": "select"},
                 {"label": "Elements", "pattern": "My Leave Entitlements"},
                 {"label": "Action", "pattern": "Click"},
                 {"label": "Element", "pattern": "Generate Button"}
            ]

ruler.add_patterns(patterns)

TRAIN_DATA = []

#iterate over the corpus again
for sentence in text:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary in index 1 of the list, so it needs to be an empty list
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
    TRAIN_DATA.append([sentence, {"entities": entities}])

print (TRAIN_DATA)

In [ ]:
import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [ ]:
convert("en", TRAIN_DATA, r"data\train.spacy")
convert("en", TRAIN_DATA, r"data\valid.spacy")

In [ ]:
!python -m spacy init fill-config data/base_config.cfg data/config.cfg

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!python -m spacy train data/config.cfg --output models/output

In [ ]:
!python -m spacy debug data data/config.cfg